In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
df=pd.read_csv('data.csv')
df.columns=['col1','col2','col3','col4','target']

In [ ]:
x=df[['col1','col2','col3','col4']].copy()
y=df['target']

In [ ]:
scaler=StandardScaler()
x[['col1','col2']]=scaler.fit_transform(x[['col1','col2']])
x['col3']=(x['col3']==130).astype(int)

In [ ]:
le=LabelEncoder()
y=le.fit_transform(y)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x.values,y,test_size=0.2,random_state=42)
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.long)
y_test_tensor=torch.tensor(y_test,dtype=torch.long)

In [ ]:
class PumpClassifier(nn.Module):
  def __init__(self):
    super(PumpClassifier,self).__init__()
    self.model=nn.Sequential(
      nn.Linear(4,64),
      nn.ReLU(),
      nn.Linear(64,128),
      nn.ReLU(),
      nn.Linear(128,64),
      nn.ReLU(),
      nn.Linear(64,20)
    )
  def forward(self,x):
    return self.model(x)

In [ ]:
model=PumpClassifier()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
n_epochs=100
for epoch in range(n_epochs):
  model.train()
  optimizer.zero_grad()
  output=model(X_train_tensor)
  loss=criterion(output,y_train_tensor)
  loss.backward()
  optimizer.step()

  model.eval()
  with torch.no_grad():
    test_output=model(X_test_tensor)
    test_pred=torch.argmax(test_output,dim=1)
    test_acc=(test_pred==y_test_tensor).float().mean()
  
  if epoch %5==0 or epoch==n_epochs-1:
    print(f"Epoch{epoch:>2} | Train Loss: {loss.item():.4f} | Test Acc:{test_acc.item():.2%}")

In [ ]:
def pumpModelPrediction(dataArray):
  """
  Cette fontion prend un vecteur ligne de 4 elements en parametre:
  - le premier element correspond à la hauteur manométrique en mètres
  - le deuxième element correspond au débit en fonctionnement de la pompe en litres par minutes
  - le troisième correspond au débit nominal de la pompe en mètres cube par heure
  - le quatrième  correspond au diamètre de la pompe en pouces.

  Le modèle actuel n'est entrainé que sur des donnés de pompes de 8 pouces de diamètre ayant un débit nominal
  de 100 ou 130 m^3/h.
  """
  df_input=pd.DataFrame([[dataArray[0],dataArray[1]]],columns=['col1','col2'])
  scaled=scaler.transform(df_input)[0]
  col3=1 if dataArray[2]==130 else 0
  col4=dataArray[3]
  features=[scaled[0],scaled[1],col3,col4]
  input_tensor=torch.tensor([features],dtype=torch.float32)

  model.eval()
  with torch.no_grad():
    output=model(input_tensor)
    pred_index=torch.argmax(output,dim=1).item()
    return le.inverse_transform([pred_index])[0]